## Import dataset 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re

In [2]:
plt.style.use('ggplot')
%matplotlib inline

In [3]:
df_all = pd.read_csv('data/april_report.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_all.columns = [col.lower().replace(' ','_') for col in df_all.columns.tolist()]
df_all.columns = [col.lower().replace('/timestamp','') for col in df_all.columns.tolist()]

In [5]:
date_col = [col for col in df_all.columns.tolist() if 'date' in col]
df_all[date_col] = df_all[date_col].apply(pd.to_datetime,format = "%m/%d/%Y %H:%M:%S")

In [6]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253306 entries, 0 to 1253305
Data columns (total 23 columns):
unique_row_id           1253306 non-null object
cvo_sid                 1253306 non-null object
event_site_id           1253306 non-null int64
touchpoint_site_id      1253306 non-null int64
event_site_name         1253306 non-null object
touchpoint_site_name    1253306 non-null object
event_type              1253306 non-null object
event_id                1253306 non-null object
event_date              1253306 non-null datetime64[ns]
touchpoint_date         1253306 non-null datetime64[ns]
total_event_value       1253306 non-null float64
touchpoint_revenue      1253306 non-null float64
conversion              1253306 non-null float64
touchpoint_type         1027589 non-null object
first_touch_date        1253306 non-null datetime64[ns]
last_touch_date         1253306 non-null datetime64[ns]
source_1                1253306 non-null object
source_2                1253306 non-nu

## Clean
* pick columns
* remove channel base


In [7]:
selected_col=['event_id','source_1','attribution_sequence','conversion','touchpoints_in_path']
#selected_col.extend(date_col)

df_prep=df_all[selected_col]

In [8]:
df_prep = df_prep[df_prep.source_1!='base']
df_prep = df_prep[df_prep.touchpoints_in_path<50]
df_prep.reset_index(drop=True, inplace=True)
df_prep.loc[df_prep.source_1.str.contains('social'),'source_1']='social'

In [9]:
def remove_small_category(df,col):
    count = df[col].value_counts()
    index = count[count<1000].index.tolist()
    return df[~df[col].isin(index)]

In [10]:
df_prep = remove_small_category(df_prep,'source_1')
# df_prep = remove_small_category(df_prep,'event_type')

# process

In [11]:
df_prep = df_prep.drop_duplicates(subset=['event_id','attribution_sequence'])

df_prep['source_sequence'] = df_prep.source_1+'_'+df_prep.attribution_sequence.astype(str)

df_conversion = df_prep.groupby('event_id',as_index=False).conversion.sum()

In [16]:
df_dummys = pd.get_dummies(df_prep,columns=['source_sequence'],prefix=[''])

df_dummys = df_dummys.drop(['source_1','attribution_sequence','conversion','touchpoints_in_path'],axis=1)

In [17]:
df_dummys_sum = df_dummys.groupby('event_id',as_index=False)[df_dummys.columns[1:]].sum()

df_dummys_sum['conversion'] = df_conversion.conversion

In [18]:
df_dummys_sum.shape

(128245, 457)

In [19]:
df_dummys_sum.head()

,event_id,_content_1,_content_10,_content_11,_content_12,_content_13,_content_14,_content_15,_content_16,_content_17,...,_video_46,_video_47,_video_48,_video_49,_video_5,_video_6,_video_7,_video_8,_video_9,conversion
0,149231181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.102254
1,149299440,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.000000
2,1491860635,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.129704
3,1491963826,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.068284
4,1491987395,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.101388
